## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Extract Social Determinants of Health](https://aws.amazon.com/marketplace/pp/prodview-hii4kdoyh4kzk)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Explain Clinical Document - Social Determinants of Health (SDOH)-Small


- **Model**: [explain_clinical_doc_sdoh_small_en](https://nlp.johnsnowlabs.com/2024/09/27/explain_clinical_doc_sdoh_small_en.html)
- **Model Description**: This pipeline extracts clinical/medical entities related to Social Determinants of Health (SDOH) from text, assigning assertion statuses to the extracted entities, and establishing relations between the extracted entities.

In [ ]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [8]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [9]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [10]:
model_name = "explain-clinical-doc-sdoh-small-en"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [ ]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [11]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [12]:
docs = [
    "Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",
    "Jane works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week."

]

sample_text = """The patient reported experiencing symptoms of anxiety and depression, which have been affecting his quality of life. 
He reported a history of childhood trauma related to violence and abuse in his household, which has contributed to his smoking, alcohol use and current mental health struggles. 
He denied any recent substance use or sexual activity and reported being monogamous in his relationship with his wife. 
The patient is an immigrant and speaks English as a second language. 
He reported difficulty accessing healthcare due to lack of medical insurance. 
He has a herniated disc, hypertension, coronary artery disease (CAD) and diabetes mellitus. 
The patient has a manic disorder, is presently psychotic and shows impulsive behavior. He has been disabled since 2001."""

### JSON

In [13]:
input_json_data = {"text": sample_text}
data = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")

ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [14]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,anxiety,46,52,Mental_Health,0.9897
1,depression,58,67,Mental_Health,0.9938
2,his,96,98,Gender,0.992
3,quality of life,100,114,Quality_Of_Life,0.6252
4,He,118,119,Gender,0.9996
5,childhood trauma,143,158,Chidhood_Event,0.74660003
6,violence,171,178,Violence_Or_Abuse,0.5394
7,abuse,184,188,Violence_Or_Abuse,0.6209
8,his,193,195,Gender,0.9536
9,his,233,235,Gender,0.9772


In [15]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,anxiety,46,52,Mental_Health,Present,0.9664
1,depression,58,67,Mental_Health,Present,0.9518
2,quality of life,100,114,Quality_Of_Life,Present,0.9941
3,violence,171,178,Violence_Or_Abuse,Past,0.9774
4,abuse,184,188,Violence_Or_Abuse,Past,0.9352
5,smoking,237,243,Smoking,Present,0.8129
6,alcohol use,246,256,Alcohol,Present,0.6393
7,mental health struggles,270,292,Mental_Health,Present,0.9152
8,sexual activity,334,348,Sexual_Activity,Absent,0.9164
9,monogamous,369,378,Sexual_Activity,Present,0.7702


In [16]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,anxiety,46,52,Mental_Health,quality of life,100,114,Quality_Of_Life,Mental_Health-Quality_Of_Life,1.0
1,depression,58,67,Mental_Health,quality of life,100,114,Quality_Of_Life,Mental_Health-Quality_Of_Life,1.0
2,violence,171,178,Violence_Or_Abuse,alcohol use,246,256,Alcohol,Violence_Or_Abuse-Alcohol,1.0
3,violence,171,178,Violence_Or_Abuse,mental health struggles,270,292,Mental_Health,Violence_Or_Abuse-Mental_Health,1.0
4,abuse,184,188,Violence_Or_Abuse,alcohol use,246,256,Alcohol,Violence_Or_Abuse-Alcohol,1.0
5,abuse,184,188,Violence_Or_Abuse,mental health struggles,270,292,Mental_Health,Violence_Or_Abuse-Mental_Health,1.0
6,smoking,237,243,Smoking,mental health struggles,270,292,Mental_Health,Smoking-Mental_Health,1.0
7,alcohol use,246,256,Alcohol,mental health struggles,270,292,Mental_Health,Alcohol-Mental_Health,1.0
8,immigrant,434,442,Population_Group,English,455,461,Language,Population_Group-Language,1.0


### JSON Lines

In [17]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [18]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "anxiety", "begin": 46, "end": 52, "ner_label": "Mental_Health", "ner_confidence": "0.9897"}, {"ner_chunk": "depression", "begin": 58, "end": 67, "ner_label": "Mental_Health", "ner_confidence": "0.9938"}, {"ner_chunk": "his", "begin": 96, "end": 98, "ner_label": "Gender", "ner_confidence": "0.992"}, {"ner_chunk": "quality of life", "begin": 100, "end": 114, "ner_label": "Quality_Of_Life", "ner_confidence": "0.6252"}, {"ner_chunk": "He", "begin": 118, "end": 119, "ner_label": "Gender", "ner_confidence": "0.9996"}, {"ner_chunk": "childhood trauma", "begin": 143, "end": 158, "ner_label": "Chidhood_Event", "ner_confidence": "0.74660003"}, {"ner_chunk": "violence", "begin": 171, "end": 178, "ner_label": "Violence_Or_Abuse", "ner_confidence": "0.5394"}, {"ner_chunk": "abuse", "begin": 184, "end": 188, "ner_label": "Violence_Or_Abuse", "ner_confidence": "0.6209"}, {"ner_chunk": "his", "begin": 193, "end": 195, "ner_label": "Gender", "ner_confidence": "0.9536

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [20]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [21]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [23]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [24]:
retrieve_json_output_from_s3(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': 'cleaning assistant',
    'begin': 7,
    'end': 24,
    'ner_label': 'Employment',
    'ner_confidence': '0.6665'},
   {'ner_chunk': 'health insurance',
    'begin': 34,
    'end': 49,
    'ner_label': 'Insurance_Status',
    'ner_confidence': '0.4379'},
   {'ner_chunk': 'alcohol abuse',
    'begin': 90,
    'end': 102,
    'ner_label': 'Alcohol',
    'ner_confidence': '0.6923'},
   {'ner_chunk': 'her',
    'begin': 110,
    'end': 112,
    'ner_label': 'Gender',
    'ner_confidence': '0.9923'},
   {'ner_chunk': 'teens',
    'begin': 114,
    'end': 118,
    'ner_label': 'Age',
    'ner_confidence': '0.7475'},
   {'ner_chunk': 'drinking',
    'begin': 121,
    'end': 128,
    'ner_label': 'Alcohol',
    'ner_confidence': '0.9493'},
   {'ner_chunk': 'beer',
    'begin': 130,
    'end': 133,
    'ner_label': 'Alcohol',
    'ner_confidence': '0.9728'},
   {'ner_chunk': 'daily',
    'begin': 135,
    'end': 139,
    'ner_label': 'Substance_Frequency',
 

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [26]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [27]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "cleaning assistant", "begin": 7, "end": 24, "ner_label": "Employment", "ner_confidence": "0.6665"}, {"ner_chunk": "health insurance", "begin": 34, "end": 49, "ner_label": "Insurance_Status", "ner_confidence": "0.4379"}, {"ner_chunk": "alcohol abuse", "begin": 90, "end": 102, "ner_label": "Alcohol", "ner_confidence": "0.6923"}, {"ner_chunk": "her", "begin": 110, "end": 112, "ner_label": "Gender", "ner_confidence": "0.9923"}, {"ner_chunk": "teens", "begin": 114, "end": 118, "ner_label": "Age", "ner_confidence": "0.7475"}, {"ner_chunk": "drinking", "begin": 121, "end": 128, "ner_label": "Alcohol", "ner_confidence": "0.9493"}, {"ner_chunk": "beer", "begin": 130, "end": 133, "ner_label": "Alcohol", "ner_confidence": "0.9728"}, {"ner_chunk": "daily", "begin": 135, "end": 139, "ner_label": "Substance_Frequency", "ner_confidence": "0.9908"}, {"ner_chunk": "30 years", "begin": 145, "end": 152, "ner_label": "Substance_Duration", "ner_confidence": "0.5964"}, {"

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

